In [ ]:
import numpy as np
import tensorflow as tf
import requests
import pandas as pd
import urllib.request
import os
import PIL
from PIL import Image
from tensorflow import keras
from matplotlib import image
from matplotlib import pyplot
from os import listdir
import cv2
from skimage.transform import resize
import matplotlib.pyplot as plt
from skimage import color
from skimage import io # load all images in a directory
from sklearn.utils import shuffle
from skimage.color import rgb2gray

In [ ]:
!git  clone https://github.com/ardamavi/Sign-Language-Digits-Dataset
!pip install Pillow

fatal: destination path 'Sign-Language-Digits-Dataset' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
grey_images= []
output=[]
for x in range(0,10):
 for filename in listdir('/content/Sign-Language-Digits-Dataset/Dataset/'+str(x)):
  #load image
  img_data = Image.open('/content/Sign-Language-Digits-Dataset/Dataset/'+str(x)+'/' + filename)
  #setting output
  output.append(x);
  #store grey image
  img=Image.open('/content/Sign-Language-Digits-Dataset/Dataset/'+str(x)+'/' + filename)
  imgGray=img.convert('L')
  grey_images.append(imgGray)
grey_images, output = shuffle(grey_images, output, random_state=42) 


In [ ]:
#resize
new_gray_images=[]
newsize = (100, 100)
print(type(grey_images))
for image in grey_images:
  new_gray_images.append(image.resize(newsize))

<class 'list'>


In [ ]:
split_1 = int(0.8 * len(grey_images))
split_2 = int(0.8 * len(output))

np.stack(new_gray_images, axis=0).astype('float32')
x_train = new_gray_images[:split_1]#80 
x_train=np.array(x_train)
y_train = np.array(output[:split_2])
y_train=np.array(y_train).flatten()
x_test = np.array(new_gray_images[split_1:])#20
x_test=np.array(x_test)
y_test =np.array(output[split_2:]).flatten()
Normalized_x_train=[]
Normalized_x_test=[]

#Normalize
print(x_train[0])
for image in x_train:
  image_array = np.asarray(image)
  normalized_array = image_array / 255
  Normalized_x_train.append(normalized_array)


for image in x_test:
  image_array = np.asarray(image)
  normalized_array = image_array / 255
  Normalized_x_test.append(normalized_array)



x_train_new=np.array(Normalized_x_train)
x_test_new=np.array(Normalized_x_test)

print(x_train_new.shape)
print(y_train.shape)
print(x_test_new.shape)
print(y_test.shape)
print(type(x_train_new))
print(type(y_train))
print(type(x_test_new))
print(type(y_test))


<ipython-input-5-f7f3cadf500e>:6: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  x_train=np.array(x_train)
<ipython-input-5-f7f3cadf500e>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train=np.array(x_train)
<ipython-input-5-f7f3cadf500e>:9: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__

<PIL.Image.Image image mode=L size=100x100 at 0x7FB268C7C460>
(1649, 100, 100)
(1649,)
(413, 100, 100)
(413,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
#Model 1
#Simple Model with No Hidden Layers
#dense means all the neurons in one layer is connected with every other neuron in another layer
#The output is 100 neurons
def model1():
  model=keras.Sequential([
      keras.layers.Flatten(input_shape=(100,100,1)),
      keras.layers.Dense(10, activation='sigmoid')
  ])
  #we used adam optmizer
  #we used this loss because our output data is categorical although we could use mean square error
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model 

In [ ]:
#Model 2
#Model with No Hidden Layers
#dense means all the neurons in one layer is connected with every other neuron in another layer
#The output is 100 neurons
def model2():
  model=keras.Sequential([
      keras.layers.Flatten(input_shape=(100,100,1)),
      keras.layers.Dense(500, activation='relu'),
      keras.layers.Dense(10, activation='softmax')
  ])
  #we used adam optmizer
  #we used this loss because our output data is categorical although we could use mean square error
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
#Model 2
#Model with No Hidden Layers
#dense means all the neurons in one layer is connected with every other neuron in another layer
#The output is 100 neurons
def model3():
  model=keras.Sequential([
      keras.layers.Flatten(input_shape=(100,100,1)),
      keras.layers.Dense(500, activation='relu'),
      keras.layers.Dense(1000, activation='relu'),
      keras.layers.Dense(10, activation='softmax')
  ])
  #we used adam optmizer
  #we used this loss because our output data is categorical although we could use mean square error
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
from sklearn.model_selection import KFold
def kFold(model, x_train,y_train):
  kfold=KFold(n_splits=5)
  for x_train_ind,x_test_ind in kfold.split(x_train,y_train):
    x_train_set, y_train_set,x_test_set,y_test_set=x_train[x_train_ind],y_train[x_train_ind],x_train[x_test_ind],y_train[x_test_ind]
    model.fit(x_train_set,y_train_set,epochs=10)
    print("*************")
    #print(model.evaluate(x_test_set,y_test_set))
    testLoss, testAccuracy = model.evaluate(x_test_set,y_test_set, verbose=2)
    print('\ntest accuracy', testAccuracy)

In [ ]:
print("-------------------Evaluate Model 1-------------------")
model_1=model1()
kFold(model_1, x_train_new,y_train)
#y_predicted = model2.predict(x_test_new)
#y_predicted[3]

-------------------Evaluate Model 1-------------------
Epoch 1/10
42/42 [==============================] - 1s 7ms/step - loss: 3.7156 - accuracy: 0.1296
Epoch 2/10
42/42 [==============================] - 0s 5ms/step - loss: 2.1037 - accuracy: 0.2904
Epoch 3/10
42/42 [==============================] - 0s 5ms/step - loss: 1.8657 - accuracy: 0.4056
Epoch 4/10
42/42 [==============================] - 0s 6ms/step - loss: 1.7833 - accuracy: 0.4230
Epoch 5/10
42/42 [==============================] - 0s 5ms/step - loss: 1.5459 - accuracy: 0.4966
Epoch 6/10
42/42 [==============================] - 0s 5ms/step - loss: 1.3853 - accuracy: 0.5497
Epoch 7/10
42/42 [==============================] - 0s 5ms/step - loss: 1.5484 - accuracy: 0.5034
Epoch 8/10
42/42 [==============================] - 0s 5ms/step - loss: 1.2088 - accuracy: 0.5989
Epoch 9/10
42/42 [==============================] - 0s 8ms/step - loss: 1.0802 - accuracy: 0.6422
Epoch 10/10
42/42 [==============================] - 0s 6ms/ste

In [ ]:
print("-------------------Evaluate Model 2-------------------")
model_2=model2()
kFold(model_2, x_train_new,y_train)

-------------------Evaluate Model 2-------------------
Epoch 1/10
42/42 [==============================] - 2s 34ms/step - loss: 8.1582 - accuracy: 0.1092
Epoch 2/10
42/42 [==============================] - 1s 34ms/step - loss: 2.1756 - accuracy: 0.2358
Epoch 3/10
42/42 [==============================] - 1s 34ms/step - loss: 1.9718 - accuracy: 0.3237
Epoch 4/10
42/42 [==============================] - 1s 33ms/step - loss: 1.7353 - accuracy: 0.3973
Epoch 5/10
42/42 [==============================] - 1s 33ms/step - loss: 1.5713 - accuracy: 0.4549
Epoch 6/10
42/42 [==============================] - 1s 33ms/step - loss: 1.4576 - accuracy: 0.5413
Epoch 7/10
42/42 [==============================] - 1s 33ms/step - loss: 1.3993 - accuracy: 0.5231
Epoch 8/10
42/42 [==============================] - 1s 33ms/step - loss: 1.3392 - accuracy: 0.5459
Epoch 9/10
42/42 [==============================] - 1s 33ms/step - loss: 1.1738 - accuracy: 0.6118
Epoch 10/10
42/42 [==============================] - 1

In [ ]:
print("-------------------Evaluate Model 3-------------------")
model_3=model3()
kFold(model_3, x_train_new,y_train)

-------------------Evaluate Model 3-------------------
Epoch 1/10
42/42 [==============================] - 3s 59ms/step - loss: 4.1072 - accuracy: 0.1039
Epoch 2/10
42/42 [==============================] - 2s 51ms/step - loss: 2.2621 - accuracy: 0.1774
Epoch 3/10
42/42 [==============================] - 2s 38ms/step - loss: 2.0299 - accuracy: 0.2502
Epoch 4/10
42/42 [==============================] - 2s 38ms/step - loss: 1.8542 - accuracy: 0.3192
Epoch 5/10
42/42 [==============================] - 2s 39ms/step - loss: 1.7875 - accuracy: 0.3199
Epoch 6/10
42/42 [==============================] - 2s 38ms/step - loss: 1.6440 - accuracy: 0.3571
Epoch 7/10
42/42 [==============================] - 2s 44ms/step - loss: 1.5315 - accuracy: 0.4155
Epoch 8/10
42/42 [==============================] - 2s 50ms/step - loss: 1.4719 - accuracy: 0.4405
Epoch 9/10
42/42 [==============================] - 2s 40ms/step - loss: 1.6822 - accuracy: 0.3768
Epoch 10/10
42/42 [==============================] - 2

In [ ]:
print("Evaluate model 1 on the test dataset")
print(model_1.evaluate(x_test_new,y_test))

Evaluate model 1 on the test dataset
13/13 [==============================] - 0s 3ms/step - loss: 0.9861 - accuracy: 0.7167
[0.9861254096031189, 0.7167070508003235]


In [ ]:
print("Evaluate model 2 on the test dataset")
print(model_2.evaluate(x_test_new,y_test))

Evaluate model 2 on the test dataset
13/13 [==============================] - 0s 13ms/step - loss: 0.9858 - accuracy: 0.7191
[0.9857746958732605, 0.7191283106803894]


In [ ]:
print("Evaluate model 3 on the test dataset")
print(model_3.evaluate(x_test_new,y_test))

Evaluate model 3 on the test dataset
13/13 [==============================] - 0s 14ms/step - loss: 1.0751 - accuracy: 0.5714
[1.0750902891159058, 0.5714285969734192]


In [ ]:
y_pred_1 = model_1.predict(x_test_new)  

13/13 [==============================] - 0s 3ms/step


In [ ]:
y_pred_2 = model_2.predict(x_test_new)  

13/13 [==============================] - 0s 12ms/step


In [ ]:
y_pred_3 = model_3.predict(x_test_new)   

13/13 [==============================] - 0s 14ms/step


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Evaluate Model 1 
print("Calculating MSE and MAE of Model 1")
# Calculate the mean squared error
mse = mean_squared_error(y_test[:5], y_pred_1[:5])
print(f'MSE: {mse:.2f}')

# Calculate the mean absolute error
mae = mean_absolute_error(y_test[:5], y_pred_1[:5])
print(f'MAE: {mae:.2f}')

Calculating MSE and MAE of Model 1
MSE: 19.83
MAE: 3.80


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Evaluate Model 1 
print("Calculating MSE and MAE of Model 2")
# Calculate the mean squared error
mse = mean_squared_error(y_test[:5], y_pred_2[:5])
print(f'MSE: {mse:.2f}')

# Calculate the mean absolute error
mae = mean_absolute_error(y_test[:5], y_pred_2[:5])
print(f'MAE: {mae:.2f}')

Calculating MSE and MAE of Model 2
MSE: 27.40
MAE: 4.60


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Evaluate Model 3
print("Calculating MSE and MAE of Model 3")
# Calculate the mean squared error
mse = mean_squared_error(y_test[:100], y_pred_3[:100])
print(f'MSE: {mse:.2f}')

# Calculate the mean absolute error
mae = mean_absolute_error(y_test[:100], y_pred_3[:100])
print(f'MAE: {mae:.2f}')

Calculating MSE and MAE of Model 3
MSE: 32.35
MAE: 4.87


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

print("----------------Evaluate Model 1----------------")

y_pred_1 = np.argmax(y_pred_1, axis=1)

# Calculate precision, recall, and f-score for each class
precision, recall, fscore, support = precision_recall_fscore_support(y_test[:100], y_pred_1[:100], average=None)

confusion_mat = confusion_matrix(y_test[:100], y_pred_1[:100])
print("Confusion Matrix")
print(confusion_mat)
print("**************************")
print("Classification Report")
print(classification_report(y_test[:100], y_pred_1[:100]))

----------------Evaluate Model 1----------------
Confusion Matrix
[[ 8  1  0  0  0  0  0  0  0  0]
 [ 1  6  0  0  0  0  0  0  1  0]
 [ 1  0  3  0  0  0  2  1  2  0]
 [ 0  0  0  5  0  1  0  2  0  1]
 [ 0  0  0  0  6  0  1  1  2  0]
 [ 0  0  0  0  1  7  0  0  0  0]
 [ 1  0  0  0  4  0  4  0  0  0]
 [ 1  0  0  0  2  0  0  5  0  0]
 [ 2  0  0  0  1  0  1  0 13  1]
 [ 1  0  0  1  0  1  0  0  0  9]]
**************************
Classification Report
              precision    recall  f1-score   support

           0       0.53      0.89      0.67         9
           1       0.86      0.75      0.80         8
           2       1.00      0.33      0.50         9
           3       0.83      0.56      0.67         9
           4       0.43      0.60      0.50        10
           5       0.78      0.88      0.82         8
           6       0.50      0.44      0.47         9
           7       0.56      0.62      0.59         8
           8       0.72      0.72      0.72        18
           9 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

print("----------------Evaluate Model 2----------------")

y_pred_2 = np.argmax(y_pred_2, axis=1)

# Calculate precision, recall, and f-score for each class
precision, recall, fscore, support = precision_recall_fscore_support(y_test[:100], y_pred_2[:100], average=None)

confusion_mat = confusion_matrix(y_test[:100], y_pred_2[:100])
print("Confusion Matrix")
print(confusion_mat)
print("**************************")
print("Classification Report")
print(classification_report(y_test[:100], y_pred_2[:100]))

----------------Evaluate Model 2----------------
Confusion Matrix
[[ 8  1  0  0  0  0  0  0  0  0]
 [ 1  6  0  0  0  0  0  0  1  0]
 [ 0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  1  0  0  0  1]
 [ 1  0  0  0  6  0  0  1  2  0]
 [ 0  0  0  0  1  7  0  0  0  0]
 [ 1  0  0  0  2  0  3  0  3  0]
 [ 0  0  2  0  1  1  0  4  0  0]
 [ 2  0  0  0  1  0  0  0 15  0]
 [ 1  0  0  1  0  3  0  0  3  4]]
**************************
Classification Report
              precision    recall  f1-score   support

           0       0.57      0.89      0.70         9
           1       0.86      0.75      0.80         8
           2       0.82      1.00      0.90         9
           3       0.88      0.78      0.82         9
           4       0.55      0.60      0.57        10
           5       0.58      0.88      0.70         8
           6       1.00      0.33      0.50         9
           7       0.80      0.50      0.62         8
           8       0.62      0.83      0.71        18
           9 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report
print("----------------Evaluate Model 3----------------")

y_pred_3 = np.argmax(y_pred_3, axis=1)

# Calculate precision, recall, and f-score for each class
precision, recall, fscore, support = precision_recall_fscore_support(y_test[:100], y_pred_3[:100], average=None)

confusion_mat = confusion_matrix(y_test[:100], y_pred_3[:100])
print("Confusion Matrix")
print(confusion_mat)
print("**************************")
print("Classification Report")
print(classification_report(y_test[:100], y_pred_3[:100]))

----------------Evaluate Model 3----------------
Confusion Matrix
[[8 1 0 0 0 0 0 0 0 0]
 [1 6 1 0 0 0 0 0 0 0]
 [0 3 5 0 0 0 0 1 0 0]
 [0 0 1 6 0 1 1 0 0 0]
 [0 0 0 0 7 0 2 1 0 0]
 [0 0 0 0 1 7 0 0 0 0]
 [0 0 1 0 0 0 4 3 1 0]
 [0 1 2 0 0 0 2 1 2 0]
 [1 0 0 0 8 0 3 0 5 1]
 [0 0 0 0 2 4 0 0 0 6]]
**************************
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.89      0.84         9
           1       0.55      0.75      0.63         8
           2       0.50      0.56      0.53         9
           3       1.00      0.67      0.80         9
           4       0.39      0.70      0.50        10
           5       0.58      0.88      0.70         8
           6       0.33      0.44      0.38         9
           7       0.17      0.12      0.14         8
           8       0.62      0.28      0.38        18
           9       0.86      0.50      0.63        12

    accuracy                           0.55       100
   m